In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="my-mistral-model",
    messages=[
        {"role": "user", "content": "Tell me a short story about king and queen"}
    ]
)

print(completion.choices[0].message)

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1"
)

completion = client.chat.completions.create(
    model="my-mistral-model",
    messages=[
        {"role": "user", "content": "Answer with just city name. What i capital of india? "}
    ]
)

print(completion.choices[0].message)

In [ ]:
import json
import random
from openai import OpenAI

# Load ShareGPT data
with open("//home/skamalj/dev/vllm/ShareGPT_V3_unfiltered_cleaned_split.json", "r") as f:
    data = json.load(f)

# Flatten and filter valid user-only prompts
def extract_user_prompts(convo):
    return [msg["value"] for msg in convo["conversations"] if msg["from"] == "human"]

# Sample 100 random prompts
random_samples = random.sample(data, 25)

# Prepare OpenAI client (local server)
client = OpenAI(base_url="http://localhost:8000/v1")

# Iterate over samples
for idx, sample in enumerate(random_samples):
    prompts = extract_user_prompts(sample)
    if not prompts:
        continue

    # Take only the **first** user prompt per sample
    first_prompt = prompts[0].strip()
    print(f"Processing sample #{idx + 1} with prompt: {first_prompt}")

    try:
        completion = client.chat.completions.create(
            model="my-mistral-model",
            messages=[{"role": "user", "content": first_prompt}]
        )

        print(f"Sample #{idx + 1}")
        print("Prompt:", first_prompt)
        print("Response:", completion.choices[0].message.content.strip())
        print("-" * 50)

    except Exception as e:
        print(f"Error on sample #{idx + 1}: {e}")


In [7]:
import json
import random
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load ShareGPT data
with open("/home/skamalj/dev/vllm/ShareGPT_V3_unfiltered_cleaned_split.json", "r") as f:
    data = json.load(f)

# Flatten and filter valid user-only prompts
def extract_user_prompts(convo):
    return [msg["value"] for msg in convo["conversations"] if msg["from"] == "human"]

# Sample 100 random prompts
random_samples = random.sample(data, 100)
client = OpenAI(base_url="http://localhost:8000/v1")

# Prepare request payloads
requests = []
for sample in random_samples:
    prompts = extract_user_prompts(sample)
    if prompts:
        requests.append(prompts[0].strip())

# Function to run a single request
def run_request(idx, prompt):
    try:
        completion = client.chat.completions.create(
            model="my-mistral-model",
            messages=[{"role": "user", "content": prompt}]
        )
        return idx, prompt, completion.choices[0].message.content.strip()
    except Exception as e:
        return idx, prompt, f"ERROR: {e}"

# Run in parallel to trigger batching in vLLM
with ThreadPoolExecutor(max_workers=32) as executor:
    futures = [executor.submit(run_request, idx, prompt) for idx, prompt in enumerate(requests)]
    for future in as_completed(futures):
        idx, prompt, response = future.result()
        print(f"Sample #{idx + 1}")
        print("Prompt:", prompt)
        print("Response:", response)
        print("-" * 50)


Sample #3
Prompt: You are an advanced text adventure game. You will receive an action as input and you will respond with a response of that action. If traveling to another room you will also include the room name and description in your response along with visible exits. 

Obvious exits could be the various cardinal directions including NW, SE, SW, and SE.

Items should be gettable and should be placed in an inventory system. Some items may be discovered by examining other items or by solving puzzles. Puzzles may involve needing one or more multiple items. 

If I want to give you further instruction aside from my actions I will prefix them with > 

Do not make any assumptions about the player's state of mind or which direction they may be facing or which direction they may have come from. 

Please generate a game with a puzzle. The game will start in a large field. The player must find their way to an abandoned government facility where a computer system can be used to save humanity. Y